<a href="https://colab.research.google.com/github/sohan-shingade/free-code-camp-machine-learning/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.layers.experimental import preprocessing


import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('expenses')
test_labels = test_features.pop('expenses')

In [ ]:
for col_name in train_dataset.columns:
    if(train_dataset[col_name].dtype == 'object'):
        train_dataset[col_name]= train_dataset[col_name].astype('category')
        train_dataset[col_name] = train_dataset[col_name].cat.codes
for col_name in test_dataset.columns:
    if(test_dataset[col_name].dtype == 'object'):
        test_dataset[col_name]= test_dataset[col_name].astype('category')
        test_dataset[col_name] = test_dataset[col_name].cat.codes

In [ ]:
normalize = preprocessing.Normalization()
normalize.adapt(np.array(train_features))


In [ ]:
tf_dataset = tf.data.Dataset.from_tensor_slices((train_dataset.values, train_labels.values))
for feat, targ in tf_dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

In [ ]:
tf_train_dataset = tf_dataset.shuffle(len(train_dataset)).batch(1)

In [ ]:
def get_compiled_model():
  model = tf.keras.Sequential([
    layers.Input((len(train_dataset.keys()),)),              
    layers.BatchNormalization(),
    layers.Dense(64),
    layers.Dense(64),
    layers.Dense(units=1)
  ])

  model.compile(optimizer=tf.optimizers.RMSprop(),
                loss=tf.losses.MeanAbsoluteError(),
                metrics=['mae', 'mse']
                )
  return model

In [ ]:
model = get_compiled_model()
hist = model.fit(train_dataset, train_labels, epochs=100)



In [ ]:
test_predictions = model.predict([test_dataset.values[0]]).flatten()
test_predictions

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
